## Lab 6: SQL


### Starting Up SQL

Before we look at SQL syntax in detail, let's first get ourselves set up to run SQL queries in Jupyter.

#### Approach #1: SQL Magic

**1. Load the `sql` Module.** 

Load `%%sql` cell magic.

In [2]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


**2. Connect to a database.**  

Here, we connect to the SQLite database `basic_examples.db`.

In [3]:
%%sql
sqlite:///data/basic_examples.db

<br/>

**3. Run a simple SQL query.** 

Note the `%%sql` lets Jupyter parse the rest of the lines as a SQL command.

In [4]:
%%sql
SELECT * FROM Dragon;

 * sqlite:///data/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


**Simple query, this time on two different lines.**

In [5]:
%%sql
SELECT *
FROM Dragon;

 * sqlite:///data/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


#### Approach #2: `pd.read_sql`

It turns out that `pandas` has a special-purpose function to parse SQL queries. We can pass in a SQL query as a string to return a `pandas` DataFrame. To achieve the same result as we did using cell magic above, we can do the following.

**1. Connect to a database**

In [6]:
import sqlalchemy 
import pandas as pd

engine = sqlalchemy.create_engine("sqlite:///data/basic_examples.db")
connection = engine.connect()

**2. Run a simple SQL query**

In [7]:
query = """
SELECT * 
FROM Dragon;
"""

pd.read_sql(query, engine)

,name,year,cute
0,hiccup,2010,10.0
1,drogon,2011,-100.0
2,dragon 2,2019,0.0
3,puff,2010,100.0
4,smaug,2011,NaN


## Tables and Schema

A **database** contains a collection of SQL **tables**. Let's connect to our "toy" database `basic_examples.db` and explore the tables it stores.

In [8]:
%%sql
sqlite:///data/basic_examples.db

In [9]:
%%sql
SELECT * FROM sqlite_master WHERE type='table'

 * sqlite:///data/basic_examples.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,7,"CREATE TABLE sqlite_sequence(name,seq)"
table,Dragon,Dragon,2,"CREATE TABLE Dragon ( name TEXT PRIMARY KEY, year INTEGER CHECK (year >= 2000), cute INTEGER)"
table,Dish,Dish,4,"CREATE TABLE Dish ( name TEXT PRIMARY KEY, type TEXT, cost INTEGER CHECK (cost >= 0))"
table,Scene,Scene,6,"CREATE TABLE Scene ( id INTEGER PRIMARY KEY AUTOINCREMENT, biome TEXT NOT NULL, city TEXT NOT NULL, visitors INTEGER CHECK (visitors >= 0), created_at DATETIME DEFAULT (DATETIME('now')))"


## Basic Queries

Every SQL query *must* contain a `SELECT` and `FROM` clause.

* `SELECT`: specify the column(s) to return in the output
* `FROM`: specify the database table from which to extract data

In [10]:
%%sql
SELECT * FROM Dragon;

 * sqlite:///data/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


In [11]:
%%sql
SELECT cute, year FROM Dragon;

 * sqlite:///data/basic_examples.db
Done.


cute,year
10,2010
-100,2011
0,2019
100,2010
None,2011


**Aliasing** with `AS`

In [12]:
%%sql
SELECT cute AS cuteness,
       year AS birth
FROM Dragon;

 * sqlite:///data/basic_examples.db
Done.


cuteness,birth
10,2010
-100,2011
0,2019
100,2010
None,2011


**Uniqueness** with `DISTINCT`

In [13]:
%%sql
SELECT DISTINCT year
FROM Dragon;

 * sqlite:///data/basic_examples.db
Done.


year
2010
2011
2019


**Filtering** with `WHERE`

In [14]:
%%sql
SELECT name, year
FROM Dragon
WHERE cute > 0;

 * sqlite:///data/basic_examples.db
Done.


name,year
hiccup,2010
puff,2010


In [15]:
%%sql
SELECT name, cute, year
FROM Dragon
WHERE cute > 0 OR year > 2013;

 * sqlite:///data/basic_examples.db
Done.


name,cute,year
hiccup,10,2010
dragon 2,0,2019
puff,100,2010


In [16]:
%%sql
SELECT name, year
FROM Dragon 
WHERE name IN ("puff", "hiccup");

 * sqlite:///data/basic_examples.db
Done.


name,year
hiccup,2010
puff,2010


In [17]:
%%sql
SELECT name, cute
FROM Dragon
WHERE cute IS NOT NULL;

 * sqlite:///data/basic_examples.db
Done.


name,cute
hiccup,10
drogon,-100
dragon 2,0
puff,100


**Ordering** data using `ORDER BY`

In [18]:
%%sql
SELECT *
FROM Dragon
ORDER BY cute DESC;

 * sqlite:///data/basic_examples.db
Done.


name,year,cute
puff,2010,100
hiccup,2010,10
dragon 2,2019,0
drogon,2011,-100
smaug,2011,None


**Restricting** output with `LIMIT` and `OFFSET`

In [19]:
%%sql
SELECT *
FROM Dragon
LIMIT 2;

 * sqlite:///data/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100


In [20]:
%%sql
SELECT *
FROM Dragon
LIMIT 2
OFFSET 1;

 * sqlite:///data/basic_examples.db
Done.


name,year,cute
drogon,2011,-100
dragon 2,2019,0


## Grouping Data with `GROUP BY`

In [21]:
%%sql
SELECT *
FROM Dish;

 * sqlite:///data/basic_examples.db
Done.


name,type,cost
ravioli,entree,10
ramen,entree,13
taco,entree,7
edamame,appetizer,4
fries,appetizer,4
potsticker,appetizer,4
ice cream,dessert,5


A small note: the fact that `type` is highlighted in green below is a consequence of Jupyter assuming that we are writing Python code (where `type` is a built-in keyword). `type` does *not* have a special meaning in SQL, so the color below does not indicate any special functionality. When we run the cell, Jupyter realizes it should recognize the code as SQL. 

In [22]:
%%sql
SELECT type
FROM Dish;

 * sqlite:///data/basic_examples.db
Done.


type
entree
entree
entree
appetizer
appetizer
appetizer
dessert


In [23]:
%%sql
SELECT type
FROM Dish
GROUP BY type;

 * sqlite:///data/basic_examples.db
Done.


type
appetizer
dessert
entree


In [24]:
%%sql
SELECT type, SUM(cost)
FROM Dish
GROUP BY type;

 * sqlite:///data/basic_examples.db
Done.


type,SUM(cost)
appetizer,12
dessert,5
entree,30


In [25]:
%%sql
SELECT type, 
       SUM(cost), 
       MIN(cost),
       MAX(name)
FROM Dish
GROUP BY type;

 * sqlite:///data/basic_examples.db
Done.


type,SUM(cost),MIN(cost),MAX(name)
appetizer,12,4,potsticker
dessert,5,5,ice cream
entree,30,7,taco


In [26]:
%%sql
SELECT year, COUNT(cute)
FROM Dragon
GROUP BY year;

 * sqlite:///data/basic_examples.db
Done.


year,COUNT(cute)
2010,2
2011,1
2019,1


In [27]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

 * sqlite:///data/basic_examples.db
Done.


year,COUNT(*)
2010,2
2011,2
2019,1


## Working with the `sql` results as Python variables

By default, executing a query with a magic command produces output but doesn't save it into any Python variable:

In [28]:
%sql SELECT * FROM Dragon

 * sqlite:///data/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


This can be inconvenient if you later want to do further processing of these data in Python.

### Storing one-line `%sql` queries

For simple one-line queries, you can use IPython's ability to store the result of a magic command like `%sql` as if it were any other Python statement, and save the output to a variable:

In [29]:
dragon_table = %sql SELECT * FROM Dragon
dragon_table

 * sqlite:///data/basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


As noted above, the result of the query is a Python variable of type `ResultSet`, more specifically:

In [30]:
type(dragon_table)

sql.run.ResultSet

You need to manually convert it to a Pandas DataFrame if you want to do pandas-things with its content:

In [31]:
dragon_df = dragon_table.DataFrame()
dragon_df

,name,year,cute
0,hiccup,2010,10.0
1,drogon,2011,-100.0
2,dragon 2,2019,0.0
3,puff,2010,100.0
4,smaug,2011,NaN


You can configure `jupysql` to _automatically_ convert all outputs to Pandas DataFrames. This can be handy if you intend all your Python-side work to be done with Pandas, as it saves you from manually having to call `.DataFrame()` first on all outputs. On the other hand, you don't get access to the original SQL `ResultSet` object, which have a number of interesting properties and capabilities. You can learn more about those in the [jupysql documentation](https://jupysql.ploomber.io).

For now, let's turn this on so you can see how this simplified, "pandas all the way" worfklow looks like:

In [32]:
%config SqlMagic.autopandas = True

In [33]:
dragon_df = %sql SELECT * FROM Dragon
dragon_df

 * sqlite:///data/basic_examples.db
Done.


,name,year,cute
0,hiccup,2010,10.0
1,drogon,2011,-100.0
2,dragon 2,2019,0.0
3,puff,2010,100.0
4,smaug,2011,NaN


In [34]:
type(dragon_df)

pandas.core.frame.DataFrame

### Storing multi-line `%%sql` queries

For a more complex query that won't fit in one line, such as for example:

In [35]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

 * sqlite:///data/basic_examples.db
Done.


,year,COUNT(*)
0,2010,2
1,2011,2
2,2019,1


You can use the `variable <<` syntax in jupysql to store its output (this will honor your `autopandas` state and store either a `sql.run.ResultState` or a Pandas `DataFrame`):

In [36]:
%%sql dragon_years <<
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

 * sqlite:///data/basic_examples.db
Done.
Returning data to local variable dragon_years


In [37]:
dragon_years

,year,COUNT(*)
0,2010,2
1,2011,2
2,2019,1


### Filtering Groups Using `HAVING`

In [38]:
%%sql
SELECT type, COUNT(*)
FROM Dish
GROUP BY type
HAVING MAX(cost) < 8;

 * sqlite:///data/basic_examples.db
Done.


,type,COUNT(*)
0,appetizer,3
1,dessert,1


In [39]:
%%sql
SELECT type, MAX(name)
FROM DishDietary
WHERE notes == 'gf'
GROUP BY type
HAVING MAX(cost) <= 7;

 * sqlite:///data/basic_examples.db
(sqlite3.OperationalError) no such table: DishDietary
[SQL: SELECT type, MAX(name)
FROM DishDietary
WHERE notes == 'gf'
GROUP BY type
HAVING MAX(cost) <= 7;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


### EDA in SQL

Our typical workflow when working with "big data" is:
* Use SQL to query data from a database
* Use Python (with `pandas`) to analyze this data in detail

We can, however, still perform simple data cleaning and re-structuring using SQL directly. To do so, we'll consider the `Title` table from the IMDB dataset.

In [40]:
%%sql
sqlite:///data/imdbmini.db

In [41]:
%%sql
SELECT * FROM sqlite_master WHERE type='table'

   sqlite:///data/basic_examples.db
 * sqlite:///data/imdbmini.db
Done.


,type,name,tbl_name,rootpage,sql
0,table,Title,Title,2,"CREATE TABLE ""Title"" (\n""tconst"" INTEGER,\n ""..."
1,table,Name,Name,12,"CREATE TABLE ""Name"" (\n""nconst"" INTEGER,\n ""p..."
2,table,Role,Role,70,"CREATE TABLE ""Role"" (\ntconst INTEGER,\norderi..."
3,table,Rating,Rating,41,"CREATE TABLE ""Rating"" (\ntconst INTEGER,\naver..."


In [42]:
%%sql
SELECT *
FROM Title
ORDER BY RANDOM()
LIMIT 10;

   sqlite:///data/basic_examples.db
 * sqlite:///data/imdbmini.db
Done.


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,3104988,movie,Crazy Rich Asians,Crazy Rich Asians,0,2018,None,120,"Comedy,Drama,Romance"
1,4595882,movie,Can You Ever Forgive Me?,Can You Ever Forgive Me?,0,2018,None,106,"Biography,Comedy,Crime"
2,2473794,movie,Mr. Turner,Mr. Turner,0,2014,None,150,"Biography,Drama,History"
3,1912398,movie,God Bless America,God Bless America,0,2011,None,105,"Comedy,Crime,Drama"
4,488085,movie,Big Nothing,Big Nothing,0,2006,None,86,"Comedy,Crime,Thriller"
5,96251,movie,Tetsuo: The Iron Man,Tetsuo,0,1989,None,67,"Fantasy,Horror,Sci-Fi"
6,5719700,movie,Home Again,Home Again,0,2017,None,97,"Comedy,Drama,Romance"
7,4225622,movie,The Babysitter,The Babysitter,0,2017,None,85,"Comedy,Horror"
8,58586,movie,A Shot in the Dark,A Shot in the Dark,0,1964,None,102,"Comedy,Mystery"
9,5697572,movie,Cats,Cats,0,2019,None,110,"Comedy,Drama,Family"


#### Matching Text Using `LIKE`

In [43]:
%%sql
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE "%Star Wars%"

   sqlite:///data/basic_examples.db
 * sqlite:///data/imdbmini.db
Done.


,titleType,primaryTitle
0,movie,Star Wars: Episode IV - A New Hope
1,movie,Star Wars: Episode V - The Empire Strikes Back
2,movie,Star Wars: Episode VI - Return of the Jedi
3,movie,Star Wars: Episode I - The Phantom Menace
4,movie,Star Wars: Episode II - Attack of the Clones
5,movie,Star Wars: Episode III - Revenge of the Sith
6,tvSeries,Star Wars: Clone Wars
7,tvSeries,Star Wars: The Clone Wars
8,movie,Star Wars: The Clone Wars
9,movie,Star Wars: Episode VII - The Force Awakens


In [44]:
%%sql
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE "Harry Potter and the Deathly Hallows: Part _"

   sqlite:///data/basic_examples.db
 * sqlite:///data/imdbmini.db
Done.


,titleType,primaryTitle
0,movie,Harry Potter and the Deathly Hallows: Part 1
1,movie,Harry Potter and the Deathly Hallows: Part 2


#### Converting Data Types Using `CAST`

In [45]:
%%sql
SELECT primaryTitle, CAST(runtimeMinutes AS INT)
FROM Title
LIMIT 10;

   sqlite:///data/basic_examples.db
 * sqlite:///data/imdbmini.db
Done.


,primaryTitle,CAST(runtimeMinutes AS INT)
0,A Trip to the Moon,13
1,The Birth of a Nation,195
2,The Cabinet of Dr. Caligari,76
3,The Kid,68
4,Nosferatu,94
5,Sherlock Jr.,45
6,Battleship Potemkin,75
7,The Gold Rush,95
8,Metropolis,153
9,The General,67


### Applying Conditions With `CASE`

Here, we return a random order so we can see the various movie ages (otherwise, the top few entries happen to all be old movies).

In [46]:
%%sql
SELECT titleType, startYear,
CASE WHEN startYear < 1950 THEN "old"
     WHEN startYear < 2000 THEN "mid-aged"
     ELSE "new"
     END AS movie_age
FROM Title
ORDER BY RANDOM()
LIMIT 10;

   sqlite:///data/basic_examples.db
 * sqlite:///data/imdbmini.db
Done.


,titleType,startYear,movie_age
0,movie,1962,mid-aged
1,movie,1984,mid-aged
2,movie,1992,mid-aged
3,movie,1984,mid-aged
4,movie,1995,mid-aged
5,movie,2007,new
6,tvSeries,1998,mid-aged
7,movie,2014,new
8,movie,2016,new
9,movie,2010,new


### Joining Tables

We combine data from multiple tables by performing a **join**. We will explore joins using the cats database, which includes two tables: `s` and `t`.

In [47]:
%%sql
sqlite:///data/basic_examples_v2.db

In [48]:
%%sql
SELECT * FROM s;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,name
0,0,Apricot
1,1,Boots
2,2,Cally
3,4,Eugene


In [49]:
%%sql
SELECT * FROM t;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,breed
0,1,persian
1,2,ragdoll
2,4,bengal
3,5,persian


#### Inner Join

In [50]:
%%sql
SELECT s.id, name, breed
FROM s INNER JOIN t ON s.id = t.id;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,name,breed
0,1,Boots,persian
1,2,Cally,ragdoll
2,4,Eugene,bengal


By default, `JOIN`ing without specifying a join type will default to an inner join.

In [51]:
%%sql
SELECT s.id, name, breed
FROM s JOIN t ON s.id = t.id;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,name,breed
0,1,Boots,persian
1,2,Cally,ragdoll
2,4,Eugene,bengal


### Cross Join

In [52]:
%%sql
SELECT *
FROM s CROSS JOIN t;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,name,id,breed
0,0,Apricot,1,persian
1,0,Apricot,2,ragdoll
2,0,Apricot,4,bengal
3,0,Apricot,5,persian
4,1,Boots,1,persian
5,1,Boots,2,ragdoll
6,1,Boots,4,bengal
7,1,Boots,5,persian
8,2,Cally,1,persian
9,2,Cally,2,ragdoll


Equivalently, a cross join can be performed using the following syntax:

In [53]:
%%sql
SELECT *
FROM s, t;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,name,id,breed
0,0,Apricot,1,persian
1,0,Apricot,2,ragdoll
2,0,Apricot,4,bengal
3,0,Apricot,5,persian
4,1,Boots,1,persian
5,1,Boots,2,ragdoll
6,1,Boots,4,bengal
7,1,Boots,5,persian
8,2,Cally,1,persian
9,2,Cally,2,ragdoll


Conceptually, an inner join is equivalent to a cross join where irrelevant rows are removed.

In [54]:
%%sql
SELECT *
FROM s CROSS JOIN t
WHERE s.id = t.id;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,name,id,breed
0,1,Boots,1,persian
1,2,Cally,2,ragdoll
2,4,Eugene,4,bengal


#### Left Outer Join

In [55]:
%%sql
SELECT *
FROM s LEFT JOIN t ON s.id = t.id;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,name,id,breed
0,0,Apricot,NaN,None
1,1,Boots,1.0,persian
2,2,Cally,2.0,ragdoll
3,4,Eugene,4.0,bengal


#### Right Outer Join

In [56]:
%%sql
SELECT *
FROM s RIGHT JOIN t ON s.id = t.id;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,name,id,breed
0,1.0,Boots,1,persian
1,2.0,Cally,2,ragdoll
2,4.0,Eugene,4,bengal
3,NaN,None,5,persian


#### Full Outer Join

In [57]:
%%sql
SELECT *
FROM s FULL JOIN t ON s.id = t.id;

   sqlite:///data/basic_examples.db
 * sqlite:///data/basic_examples_v2.db
   sqlite:///data/imdbmini.db
Done.


,id,name,id,breed
0,0.0,Apricot,NaN,None
1,1.0,Boots,1.0,persian
2,2.0,Cally,2.0,ragdoll
3,4.0,Eugene,4.0,bengal
4,NaN,None,5.0,persian


#### Aliasing in Joins

Let's return to the IMDB dataset. Now, we'll consider two tables: `Title` and `Rating`.

In [58]:
%%sql
sqlite:///data/imdbmini.db

When working with tables that have long names, we often create an **alias** using the `AS` keyword (much like we did with columns in the previous lecture). This makes it easier to reference these tables when performing a join.

In [59]:
%%sql
SELECT primaryTitle, averageRating
FROM Title AS T INNER JOIN Rating AS R
ON T.tconst = R.tconst;

   sqlite:///data/basic_examples.db
   sqlite:///data/basic_examples_v2.db
 * sqlite:///data/imdbmini.db
Done.


,primaryTitle,averageRating
0,A Trip to the Moon,8.2
1,The Birth of a Nation,6.3
2,The Cabinet of Dr. Caligari,8.1
3,The Kid,8.3
4,Nosferatu,7.9
...,...,...
7569,Hollywood,7.5
7570,Holidate,6.1
7571,I Care a Lot,6.3
7572,Perfect Game,9.9


The `AS` keyword is optional – omitting it will still create an alias. We typically include `AS` to make our queries more readable.

In [60]:
%%sql
SELECT primaryTitle, averageRating
FROM Title T INNER JOIN Rating R
ON T.tconst = R.tconst;

   sqlite:///data/basic_examples.db
   sqlite:///data/basic_examples_v2.db
 * sqlite:///data/imdbmini.db
Done.


,primaryTitle,averageRating
0,A Trip to the Moon,8.2
1,The Birth of a Nation,6.3
2,The Cabinet of Dr. Caligari,8.1
3,The Kid,8.3
4,Nosferatu,7.9
...,...,...
7569,Hollywood,7.5
7570,Holidate,6.1
7571,I Care a Lot,6.3
7572,Perfect Game,9.9


Referencing columns using the full or aliased table name is important to avoid ambiguity. Suppose the tables we are trying to join both include a column with the same name, like the `tconst` columns present in both the `Title` and `Rating` tables of the IMDB database. If we do not specify which table's column we wish to reference, SQL will not be able to process our query.

In the cell below, it is unclear if we are referring to the `tconst` column from the `Title` table or the `tconst` column from the `Rating` table. SQL errors.

In [61]:
%%sql
SELECT primaryTitle, averageRating
FROM Title AS T INNER JOIN Rating AS R
ON tconst = tconst;

   sqlite:///data/basic_examples.db
   sqlite:///data/basic_examples_v2.db
 * sqlite:///data/imdbmini.db
(sqlite3.OperationalError) ambiguous column name: tconst
[SQL: SELECT primaryTitle, averageRating
FROM Title AS T INNER JOIN Rating AS R
ON tconst = tconst;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Refer to columns in the format `table_alias.column_name` to avoid ambiguity.

In [62]:
%%sql
SELECT primaryTitle, averageRating
FROM Title AS T INNER JOIN Rating AS R
ON T.tconst = R.tconst;

   sqlite:///data/basic_examples.db
   sqlite:///data/basic_examples_v2.db
 * sqlite:///data/imdbmini.db
Done.


,primaryTitle,averageRating
0,A Trip to the Moon,8.2
1,The Birth of a Nation,6.3
2,The Cabinet of Dr. Caligari,8.1
3,The Kid,8.3
4,Nosferatu,7.9
...,...,...
7569,Hollywood,7.5
7570,Holidate,6.1
7571,I Care a Lot,6.3
7572,Perfect Game,9.9


## Exercise (Submit for participation)

Convert each Pandas expression to SQL assuming data represents a Pandas DataFrame containing 3 columns: name, rank, and year. Both the rank and year are stored as integers.


In [63]:
data.1oc[data["rank"] < 10, "name"].value_counts().reset_index() 

#Hint:Remember that value counts returns a sorted output!

SyntaxError: invalid syntax (<ipython-input-63-d4ea124085b7>, line 1)

In [64]:
%%sql
SELECT name
FROM data
WHERE rank < 10
GROUP BY name
ORDER BY COUNT(*) DESC;

   sqlite:///data/basic_examples.db
   sqlite:///data/basic_examples_v2.db
 * sqlite:///data/imdbmini.db
(sqlite3.OperationalError) no such table: data
[SQL: SELECT name
FROM data
WHERE rank < 10
GROUP BY name
ORDER BY COUNT(*) DESC;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [ ]:
data.merge(data,on= "name").sort_values(by="name")

In [ ]:
%%sql
SELECT *
FROM data AS D1 INNER JOIN data AS D2
ON D1.name = D2.name
ORDER BY D1.name;

In [ ]:
data.groupby(["name","year"])\
    .filter(lambda sdf: len(sdf)>5)\
    .groupby(["name","year"])["rank"]\
    .min().reset_index().head(5)

In [ ]:
%%sql
SELECT name, year, MIN(rank)
FROM data
GROUP BY name, year
HAVING COUNT(*) > 5
LIMIT 5;